<a href="https://colab.research.google.com/github/MonicaKim89/Fire_detection/blob/main/Resnet_test_20210504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import keras 
keras.__version__

'2.4.3'

In [19]:
tf.__version__

'2.2.0'

In [2]:
import cv2
import math
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)'
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from tensorflow.keras import optimizers


IMG_SIZE = 100
NUM_EPOCHS = 20
NUM_CLASSES = 2
TRAIN_BATCH_SIZE = 77
TEST_BATCH_SIZE = 1 
opt =  tf.optimizers.Adam()


In [3]:
def create_model():
    resnet = ResNet50(include_top=False, pooling='avg', weights = None)
      #resnet.summary()
    my_new_model = Sequential()
    my_new_model.add(resnet)
    my_new_model.layers[0].trainable = True
    my_new_model.add(Dense(NUM_CLASSES, activation='softmax')) #dense 3, NUM_CLASSES = 3

  # Say no to train first layer (ResNet) model. It is already trained
    
    opt =  tf.optimizers.Adam()
   #opt바꾼거 확인해
    my_new_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return my_new_model

In [6]:
def train_model( model ):
    #ata_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

    data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                #sear_range=0.01,
                                zoom_range=[0.9, 1.25],
                                horizontal_flip=True,
                                vertical_flip=False,
                                data_format='channels_last',
                                brightness_range=[0.5, 1.5]
                               )
                                       
    train_generator = data_generator_with_aug.flow_from_directory(
            "C:\\Users\\yukir\\Documents\\Monicas_workspace\\Deeplearning\\base_dir\\train",
            target_size=(IMG_SIZE, IMG_SIZE),
            batch_size=TRAIN_BATCH_SIZE,
            class_mode='categorical')
    
   
    validation_generator = data_generator_with_aug.flow_from_directory(
            "C:\\Users\\yukir\\Documents\\Monicas_workspace\\Deeplearning\\base_dir\\test",
            target_size=(IMG_SIZE, IMG_SIZE),
            batch_size=TEST_BATCH_SIZE,
            shuffle = False,
            class_mode='categorical')
    
        
    #y_train = get_labels(train_generator)
    #weights = class_weight.compute_class_weight('balanced',np.unique(y_train), y_train)
    #dict_weights = { i: weights[i] for i in range(len(weights)) }
       
    H = model.fit(
            train_generator,
            steps_per_epoch=train_generator.n/TRAIN_BATCH_SIZE,
            epochs=NUM_EPOCHS,
            validation_data=validation_generator,
            validation_steps = 1 
            #,
            #class_weight=dict_weights
                )
    
    plot_history( H, NUM_EPOCHS )
    
    return model, train_generator,validation_generator

In [7]:
def get_label_dict(train_generator ):
# Get label to class_id mapping
    labels = (train_generator.class_indices)
    label_dict = dict((v,k) for k,v in labels.items())
    return  label_dict   

In [8]:
def get_labels( generator ):
    generator.reset()
    labels = []
    for i in range(len(generator)):
        labels.extend(np.array(generator[i][1]) )
    return np.argmax(labels, axis =1)

In [9]:
def get_pred_labels( test_generator):
    test_generator.reset()
    pred_vec=model.predict_generator(test_generator,
                                     steps=test_generator.n, #test_generator.batch_size
                                     verbose=1)
    return np.argmax( pred_vec, axis = 1), np.max(pred_vec, axis = 1)

In [10]:
def plot_history( H, NUM_EPOCHS ):
    plt.style.use("ggplot")
    fig = plt.figure()
    fig.set_size_inches(15, 5)
    
    fig.add_subplot(1, 3, 1)
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_loss"], label="val_loss")
    plt.title("Training Loss and Validation Loss on Dataset")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.legend(loc="lower left")

    
    fig.add_subplot(1, 3, 2)
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["accuracy"], label="train_accuracy")
    plt.title("Training Loss and Accuracy on Dataset")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    
    fig.add_subplot(1, 3, 3)
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, NUM_EPOCHS), H.history["val_accuracy"], label="val_accuracy")
    plt.title("Validation Loss and Accuracy on Dataset")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")


    plt.show()

In [11]:
def draw_prediction( frame, class_string ):
    x_start = frame.shape[1]- 600 # 위치
    cv2.putText(frame, class_string, (10,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    return frame

In [12]:
def prepare_image_for_prediction( img):
   
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    # The below function inserts an additional dimension at the axis position provided
    img = np.expand_dims(img, axis=0)
    # perform pre-processing that was done when resnet model was trained.
    return preprocess_input(img)

In [13]:
def get_display_string(pred_class, label_dict):
    txt = ""
    for c, confidence in pred_class:
        print(c)
        print(confidence)
        txt += label_dict[c]
        txt += '['+ str(confidence) +']'
    return txt
    # txt = neutral[0.38]

In [14]:
def predict(  model, video_path, filename, label_dict ):
    
    vs = cv2.VideoCapture(video_path)
    fps = math.floor(vs.get(cv2.CAP_PROP_FPS))
    ret_val = True
    writer = 0
    
    while True:
        ret_val, frame = vs.read()
        if not ret_val:
            break
       
        resized_frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        frame_for_pred = prepare_image_for_prediction( resized_frame )
        pred_vec = model.predict(frame_for_pred)
        print('pred_vec')
        print(pred_vec)
        pred_class =[]
        confidence = np.round(pred_vec.max(),2) 
        
        pc = pred_vec.argmax()
        pred_class.append( (pc, confidence) )

        if pred_class:
            txt = get_display_string(pred_class, label_dict)
            print('최종txt' +txt)       
            frame = draw_prediction( frame, txt )
            #print('pred_class')
            #print(pred_class)
            #print('label_dict')
            #print(label_dict)
        if not writer:
            fourcc = cv2.VideoWriter_fourcc(*"XVID")
            writer = cv2.VideoWriter(filename, fourcc, fps,(frame.shape[1], frame.shape[0]), True)
        writer.write(frame)
        
    vs.release()
    writer.release()

In [15]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 23,538,690
Non-trainable params: 53,120
_________________________________________________________________


In [16]:
trained_model,train_generator,validation_generator = train_model(model)
label_dict= get_label_dict(train_generator)
#model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#model.save('/content/drive/MyDrive/Caba2012_colab/Fire_dt/models/resnet.h5')

Found 5938 images belonging to 2 classes.
Found 744 images belonging to 2 classes.


TypeError: __array__() takes 1 positional argument but 2 were given

In [ ]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.save('/content/drive/MyDrive/Caba2012_colab/Fire_dt/models/resnet.h5')

In [ ]:
%cd  '/content/drive/MyDrive/Caba2012_colab/Fire_dt/datasets/test_videos/'

video_path = '/content/drive/MyDrive/Caba2012_colab/Fire_dt/datasets/test_videos/test_ori.mp4'
predict ( trained_model, video_path, '/content/drive/MyDrive/Caba2012_colab/Fire_dt/datasets/test_videos/test_ori_pred.mp4',  label_dict) 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
pred_vec
[[0.00943782 0.11557003 0.87499213]]
2
0.87
최종txtSmoke[0.87]
pred_vec
[[0.00895479 0.15624885 0.83479637]]
2
0.83
최종txtSmoke[0.83]
pred_vec
[[0.00887571 0.1684382  0.8226861 ]]
2
0.82
최종txtSmoke[0.82]
pred_vec
[[0.00829187 0.20461622 0.78709185]]
2
0.79
최종txtSmoke[0.79]
pred_vec
[[0.00867956 0.04326295 0.9480575 ]]
2
0.95
최종txtSmoke[0.95]
pred_vec
[[0.00863855 0.03975241 0.9516091 ]]
2
0.95
최종txtSmoke[0.95]
pred_vec
[[0.00816587 0.16221319 0.82962096]]
2
0.83
최종txtSmoke[0.83]
pred_vec
[[0.00790539 0.20200369 0.7900909 ]]
2
0.79
최종txtSmoke[0.79]
pred_vec
[[0.00953186 0.09127977 0.89918834]]
2
0.9
최종txtSmoke[0.9]
pred_vec
[[0.00910544 0.1375119  0.85338265]]
2
0.85
최종txtSmoke[0.85]
pred_vec
[[0.00905415 0.13924967 0.8516962 ]]
2
0.85
최종txtSmoke[0.85]
pred_vec
[[0.00978919 0.14400366 0.84620714]]
2
0.85
최종txtSmoke[0.85]
pred_vec
[[0.00935342 0.06856406 0.92208254]]
2
0.92
최종txtSmoke[0.92]
pred_vec
[[0.00942152 0.06060768 0.9299708 ]]
2
0.93
최종t